# Colliding Spheres
This Notebook launches a condition number study for the colliding spheres testcase as presented in the paper "Cell agglomeration strategy for eXtended discontinous Galerkin methods" (2024) 
 The calculations can be very long, in order to use a HPC cluster ``..\.BoSSS\etc\BatchProcessorConfig.json`` must be modified for the execution queue.

In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "CollidingSpheres2D" : buildname;
dbname = String.IsNullOrEmpty(dbname)? "condStudyBDF2" : dbname; 
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = ExecutionQueues[4];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

In [ ]:
myDB.Sessions

In [6]:
int SpaceDim = 2; //3
double _partRad = 0.6;
var Gshape = Shape.CollidingSpheres;

In [7]:
// set parameterz
int[] core_sweep = { 1};
int[] PolyDegS = new int[] {1, 2, 3};
int[] ResArray = new int[] {32};
double[] aggThresolds = new double[]  {0, 0.1, 0.2, 0.3, 0.4, 0.5};

In [8]:
int NoOfTimeSteps = 80;
bool Steady = false;
bool IncludeConvection = true;

# Grid

In [9]:
double xMax = 2, yMax = 1, zMax = 1;
double xMin = -2, yMin = -1, zMin = -1;


In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();

foreach (var Res in ResArray) {
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res * 2 + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);    
    GridCommons grd;
    string gname = SpaceDim == 2 ? "CollidingSphere2D_1BCs_longDomX" : "CollidingSphere3D_1BCs_longDomX";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;

            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells == (_xNodes.Length - 1) * (_yNodes.Length - 1) * (SpaceDim == 3 ? GenericBlas.Linspace(zMin, zMax, Res + 1).Length - 1 : 1);
            if(IsMatch) 
            tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }

    Console.WriteLine("Found {0} grids for Res={1}", tmp.Count, Res);
    if (tmp.Any()) {
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.First());
    } else {
        if (SpaceDim == 3) {
            grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
        } else {
            grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);
        }

        grd.Name = gname;
        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
        myDB.SaveGrid(ref grd);
        Grids.Add(grd);
    }
}

Console.WriteLine("\nThe grids to be simulated:");
Grids


## Generate Control object

### governing equations
- incompressible transient Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t) = \vec{u}_{body} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$
<br> $p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 
<br>where:
 $$
	\vec{u}_{body}(\vec{x},t) =
	 \begin{cases}
		\vec{u}_{g,\mathrm{L}} =  (u_\mathrm{s},0,0) \phantom{-} \quad \quad \text{on } \ \vec{x} \, | \, \psi_\mathrm{L}(\vec{x},t)>0 \text{ and } \psi_\mathrm{R}(\vec{x},t)<0, \\
		\vec{u}_{g,\mathrm{R}} =  (-u_\mathrm{s},0,0)  \quad \quad \text{on } \ \vec{x} \, | \, \psi_\mathrm{L}(\vec{x},t)<0 \text{ and } \psi_\mathrm{R}(\vec{x},t)>0, \\
		\vec{u}_{g,\mathrm{LR}} =  \vec{0} \phantom{(-u,0,)} \quad \quad  \text{on } \ \vec{x} \, | \, \psi_\mathrm{L}(\vec{x},t) \geq 0 \text{ and } \psi_\mathrm{R}(\vec{x},t) \geq 0.
	\end{cases}
$$
-while $u_\mathrm{s}=\frac{Re*\mu_A}{\rho_A*(2*r_p)}$

In [11]:
Func<IGridInfo, int,double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = 
delegate(IGridInfo grd, int k, double viscosity, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;


    //Session Settings
    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    C.DynamicLoadBalancing_On = false;
    //C.DynamicLoadBalancing_RedistributeAtStartup = true; 
    //C.DynamicLoadBalancing_Period = 10;
    //C.DynamicLoadBalancing_ImbalanceThreshold = 0.1; 

    // Physical Parameters
    // =================== 
    const double rhoA = 1;
    const double Re = 1000;
    double muA = viscosity;   
    double partRad = _partRad;
    double partDia = 2*partRad;
    double VelocitySphere = Re*muA/rhoA/partDia;
    double ts = 3*partRad/VelocitySphere/NoOfTimeSteps; 
    double[] pos = new double[SpaceDim];    
    //string rotAxis= RotAxis ?? "z";

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    // Meta data
    // ============    
    C.SessionName = string.Format("J{0}_k{1}_agg{2}_AMR{3}_LB{4}_vel{5:f2}_r{6}", J, k,AggThrs,AMR,LoadBalance,VelocitySphere,partRad);
    C.SessionName += $"_{SpaceDim}D";

    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees and configuration
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 8));
    C.saveperiod = 10;
    C.GridGuid = grd.ID;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;
    C.ImmediatePlotPeriod = 5;
    C.SuperSampling = 2; 

    // rigid body and boundary conditions
    // ===================
    double rateOfRadius = 3/(double)NoOfTimeSteps/ts; //utilized to describe sphere pos + vel
    double anglev = 0.0;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim, rateOfRadius: rateOfRadius); 
    C.SessionName += $"_rateOfRadius{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    //C.Rigidbody.SetRotationAxis(rotAxis);
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;

    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");

    // solver options
    // ===================
    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None; // idle level set (used for evolving LS)
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 0;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;
    C.AdaptiveMeshRefinement = AMR;

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = Double.MaxValue;
        C.NoOfTimesteps = 1;
        C.SessionName += "_Steady";
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps; 
    }

    C.TimeSteppingScheme = TimeSteppingScheme.BDF2; //BD4
    C.dtFixed = dt;
    C.SessionName += $"_{(TimeSteppingScheme)C.TimeSteppingScheme}_ts{NoOfTimeSteps}"; // $"_impEuler_ts{NoOfTimeSteps}";

    // Solver On-Off
    C.SkipSolveAndEvaluateResidual = false;
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;

    // Condition number study
    // ============
    var config = new OperatorAnalysisConfig();
    config.CalculateMassMatrix = true;
    config.CalculateGlobalConditionNumbers = true;
    config.CalculateStencilConditionNumbers = false;
    config.PlotStencilConditionNumbers = false;
    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
    C.SessionName += "_condStudy";

    return C;    
};

In [ ]:
var ViscositySweep = new double[]{1E-2};
var axes = new string[]{"z"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};

//var TimeSchemes = new TimeSteppingScheme[] {TimeSteppingScheme.ImplicitEuler, TimeSteppingScheme.BDF4};
List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids)
    foreach(int k in PolyDegS)
            foreach(double v in ViscositySweep)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
            {
                controls.Add(GenXNSECtrl(grd,k,v,axis,AMR,LoadBalance,agg));
            }


Console.WriteLine($"Total Number of Control Files: {controls.Count}");
controls.Summary()

In [ ]:
int k = 0;
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores;
                    var aJob   = new Job(""+Gshape+ctrl.SessionName,typeof(XNSE));

            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            aJob.UseComputeNodesExclusive = true;
            aJob.NumberOfThreads = 4;
            //aJob.ExecutionTime            = "71:59:00";
            //List<string> Cmdtmp = myBatch.AdditionalBatchCommands.ToList();
            //int NoOfNodes = 1;
            //Cmdtmp.Add($"#SBATCH -N {NoOfNodes}");
            //myBatch.AdditionalBatchCommands = Cmdtmp.ToArray();

            aJob.Activate(myBatch);
            Console.WriteLine("Submitted {0}th \n",k) ;
            k++;           
        }       
    }
